In [1]:
import requests
import pandas as pd
import re 
import csv
import time
from datetime import datetime
import mysql.connector
import pymysql, os, json
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine


In [2]:
now = datetime.today()

In [3]:
list_province_in_thai = []
list_district_in_thai = []
list_subdistrict_in_thai = []
label = pd.read_excel('../code/geocodelabel.xlsx', index_col=None)
for i in range(len(label)):
    list_province_in_thai.append(label['name_th'][i])
    list_district_in_thai.append(label['district_name_th'][i])
    list_subdistrict_in_thai.append(label['subdistrict_name_th'][i])

# clear dup
list_province_in_thai = list(dict.fromkeys(list_province_in_thai))
list_district_in_thai = list(dict.fromkeys(list_district_in_thai))
list_subdistrict_in_thai = list(dict.fromkeys(list_subdistrict_in_thai))


In [4]:
def getSevenBySearchKeyword():
    url = "https://corporate.tops.co.th:8080/get-all-store?"
    headers = {'Accept': 'application/json, text/plain, */*'}
    params = {"store_format": "familymart" }
    response = requests.get(url, headers=headers , params=params)
    #response = requests.post(url, headers=headers)
    payload = response.json()
    with open(f"../trash/familymart{str(now.date())}.json"  , "w") as outfile:
        json.dump(payload, outfile)


In [5]:
def longdoAddress(lat,lng):
    url = "https://api.longdo.com/map/services/address?"
    params = {'lat': lat , 'lon' : lng , 'key' : '1917e02b4ab264ba547fecdd6e82b4aa'}
    response =requests.get(url, params=params)
    payload = response.json()
    #print("------------------------------------------")
    #print(payload)
    #print("------------------------------------------")
    province = removeWrongkeyword(payload['province'])
    district = removeWrongkeyword(payload['district'])
    subdistrict = removeWrongkeyword(payload['subdistrict'])
    try:
        postcode = payload['postcode']
    except :
        postcode = ""
    
    time.sleep(1)
    
    return province,district,subdistrict,postcode

In [6]:
def extracteachDetail(word):
    payload_word = word
    if(re.findall(r"\จังหวัด",payload_word)):
        payload_word = re.sub("จังหวัด","",payload_word)
    elif(re.findall(r"\อำเภอ",payload_word)):
        payload_word = re.sub("อำเภอ","",payload_word)
    elif(re.findall(r"\ตำบล",payload_word)):
        payload_word = re.sub("ตำบล","",payload_word)
    elif(re.findall(r"\เขต",payload_word)):
        payload_word = re.sub("เขต","",payload_word)
    elif(re.findall(r"\แขวง",payload_word)):
        payload_word = re.sub("แขวง","",payload_word)
        
    return payload_word

In [7]:
def removeWrongkeyword(word):
    payload_word = word
    if(re.findall(r"\จ+[.]",word)):
        payload_word = re.sub(r"\จ+[.]","จังหวัด",word)
    elif(re.findall(r"\อ+[.]",word)):
        payload_word = re.sub(r"\อ+[.]","อำเภอ",word)
    elif(re.findall(r"\ต+[.]",word)):
        payload_word = re.sub(r"\ต+[.]","ตำบล",word)
    return payload_word
        

In [8]:
getSevenBySearchKeyword()

In [9]:
def getAddressDetail(address,lat,lng):
    
    str_address =  re.sub(r'["]',"",address)
    str_address = str_address.replace("  "," ")
    
    length = len(str_address)
    
    #print("index of: " + str(i))
    
    #print(lat)
    #print(lng)
    #print(str_address)
    
    #remove newline 
    if("\n" in str_address) :
        str_address = re.sub('\r','',str_address)
        str_address = re.sub('\n',' ',str_address)
    
    temp_fix = str_address.split()
    
    print(str_address)
    print(temp_fix)

    
    payload_address = ""
    
    if(length == 0):
        #print(length)
        province,district,subdistrict,postcode = longdoAddress(lat,lng)
        temp_fix.append(subdistrict)
        temp_fix.append(district)
        temp_fix.append(province)
        temp_fix.append(postcode)
        #print(temp_fix)
        time.sleep(1)
    
    else :
        
        if(not re.findall("[0-9]", temp_fix[-1])) :

            temp_fix[-1] = removeWrongkeyword(temp_fix[-1])
            temp_fix[-2] = removeWrongkeyword(temp_fix[-2])
            temp_fix[-3] = removeWrongkeyword(temp_fix[-3])
            
            province = extracteachDetail(temp_fix[-1])
            district = extracteachDetail(temp_fix[-2])
            subdistrict = extracteachDetail(temp_fix[-3])
            
        else :
            
            #province with check special case
            if re.findall("กทม.", temp_fix[-2]):
                temp_fix[-2] = temp_fix[-2].replace("กทม.","จังหวัดกรุงเทพมหานคร") 
            elif re.findall("กรุงเทพฯ", temp_fix[-2]):
                temp_fix[-2] = temp_fix[-2].replace("กรุงเทพฯ","จังหวัดกรุงเทพมหานคร")
            elif re.findall("กรุงเทพ", temp_fix[-2]):
                temp_fix[-2] = temp_fix[-2].replace("กรุงเทพ","จังหวัดกรุงเทพมหานคร") 
            
            temp_fix[-2] = removeWrongkeyword(temp_fix[-2])
            
            #district with check special case
            if re.findall("อำแภอ", temp_fix[-3]):
                temp_fix[-3] = temp_fix[-3].replace("อำแภอ","อำเภอ") 
            if re.findall("อำเภสัตหีบ", temp_fix[-3]):
                temp_fix[-3] = temp_fix[-3].replace("อำเภสัตหีบ","อำเภอสัตหีบ") 
                
            temp_fix[-3] = removeWrongkeyword(temp_fix[-3])
            
            #subdistrict with check special case
            if re.findall("อำเภอ", temp_fix[-4]) and len(temp_fix[-4]) == 5 :
                temp_fix.remove(temp_fix[-4])
            if re.findall("อ.", temp_fix[-4]) and len(temp_fix[-4]) <= 5 :
                temp_fix.remove(temp_fix[-4])
            if re.findall("ตำบลุ่งควายกิน", temp_fix[-4]) :
                temp_fix[-4] = temp_fix[-4].replace("ตำบลุ่งควายกิน","ตำบลทุ่งควายกิน")
        
            temp_fix[-4] = removeWrongkeyword(temp_fix[-4])
            
            
            if(not re.findall(r"\จังหวัด", temp_fix[-2])):
                 temp_fix[-2] = "จังหวัด" +  temp_fix[-2]
                    
            if(temp_fix[-2] == "จังหวัดกรุงเทพมหานคร"):
                if(not re.findall(r"\เขต",temp_fix[-3])):
                    temp_fix[-3] = "เขต" + temp_fix[-3]
            else:
                if(not re.findall(r"\อำเภอ",temp_fix[-3])):
                    temp_fix[-3] = "อำเภอ" + temp_fix[-3]
            
            if(temp_fix[-2] == "จังหวัดกรุงเทพมหานคร"):
                if(not re.findall(r"\ตำบล",temp_fix[-4])):
                    temp_fix[-4] = "ตำบล" + temp_fix[-4]
            else:
                if(not re.findall(r"\แขวง",temp_fix[-4])):
                    temp_fix[-4] = "แขวง" + temp_fix[-4]
   
     
            postcode = extracteachDetail(temp_fix[-1])
            province = extracteachDetail(temp_fix[-2])
            district = extracteachDetail(temp_fix[-3])
            subdistrict = extracteachDetail(temp_fix[-4])
  
    for add in temp_fix :
        payload_address = payload_address + add + " "
     
    
    is_find = False
    
    if(province == "กรุงเทพฯ"):
        province = province.replace("กรุงเทพฯ","กรุงเทพมหานคร")
    if(province not in list_province_in_thai):
        print(str_address)
        is_find = True
        province,district,subdistrict,postcode = longdoAddress(lat,lng)
        postcode = extracteachDetail(postcode)
        province = extracteachDetail(province)
        district = extracteachDetail(district)
        subdistrict = extracteachDetail(subdistrict)
    if(district not in list_district_in_thai):
        if(not is_find):
            print(str_address)
            is_find = True
            province,district,subdistrict,postcode = longdoAddress(lat,lng)
            postcode = extracteachDetail(postcode)
            province = extracteachDetail(province)
            district = extracteachDetail(district)
            subdistrict = extracteachDetail(subdistrict)
    if(subdistrict not in subdistrict):
        if(not is_find):
            print(str_address)
            is_find = True
            province,district,subdistrict,postcode = longdoAddress(lat,lng)
            postcode = extracteachDetail(postcode)
            province = extracteachDetail(province)
            district = extracteachDetail(district)
            subdistrict = extracteachDetail(subdistrict)
    '''
    print(payload_address)
    print("index of: " + str(i))
    print("subdistrict: " + subdistrict)
    print("district: " + district)
    print("province: " + province)
    print("postcode: " + postcode)
    print("------------------------------------")
    '''
    
    return payload_address,province,district,subdistrict,postcode
     

In [10]:
pro = open(f'../trash/familymart{str(now.date())}.json')
payload = json.load(pro)

In [11]:
length = len(payload)
i = 0
list_payload = []

while i < length :
    name = payload[i]['value']['store_name_th']
    name_en = payload[i]['value']['store_name']
    store_id = payload[i]['value']['internal_store_no']
    address = payload[i]['value']['location']
    address_en = payload[i]['value']['location']
    lat = payload[i]['value']['latitude']
    lng = payload[i]['value']['longitude']
    store_type_name = payload[i]['value']['store_format']
    
    if(lat != "" or lng != ""):
        payload_address,province,district,subdistrict,postcode = getAddressDetail(address,lat,lng)
        
        store_type = "Convenience Stores"
        
        if not re.findall("[ก-ฮ]", name):
            name = name_en
        
        dict_payload = { "name": name,
                        "store_id" : store_id,
                        "name_store" : store_type_name, 
                        "store_type" : store_type ,
                        "address" : payload_address ,
                        "lat" : lat,
                        "lng" : lng,
                        "subdistrict": subdistrict ,
                        "district": district,
                        "province": province,
        }
        
        list_payload.append(dict_payload)
        
    i = i + 1
    
if(store_type_name == "FamilyMart"):
    with open(f"../trash/familymart{str(now.date())}.json","w") as outfile:
            json.dump(list_payload, outfile)


19/8 หมู่ที่ 6 ตำบลบางเมืองใหม่ อำเภอเมืองสมุทรปราการ จังหวัดสมุทรปราการ 10270
['19/8', 'หมู่ที่', '6', 'ตำบลบางเมืองใหม่', 'อำเภอเมืองสมุทรปราการ', 'จังหวัดสมุทรปราการ', '10270']
4/43 ม. 1 ต.บางโฉลง อ.บางพลี จ.สมุทรปราการ 10540
['4/43', 'ม.', '1', 'ต.บางโฉลง', 'อ.บางพลี', 'จ.สมุทรปราการ', '10540']
83/64 หมู่ที่ 3 ถนนเลี่ยงเมือง ตำบลมะขามเตี้ย อำเภอเมืองสุราษฎร์ธานี จังหวัดสุราษฎร์ธานี 84000
['83/64', 'หมู่ที่', '3', 'ถนนเลี่ยงเมือง', 'ตำบลมะขามเตี้ย', 'อำเภอเมืองสุราษฎร์ธานี', 'จังหวัดสุราษฎร์ธานี', '84000']
65/1 ม.17 แขวงศาลาธรรมสพน์ เขตทวีวัฒนา จ.กรุงเทพ 10170
['65/1', 'ม.17', 'แขวงศาลาธรรมสพน์', 'เขตทวีวัฒนา', 'จ.กรุงเทพ', '10170']
65/1 ม.17 แขวงศาลาธรรมสพน์ เขตทวีวัฒนา จ.กรุงเทพ 10170
148/53,148/54 หมู่ที่ 14 ตำบลในเมือง อำเภอเมืองขอนแก่น จังหวัดขอนแก่น 40000
['148/53,148/54', 'หมู่ที่', '14', 'ตำบลในเมือง', 'อำเภอเมืองขอนแก่น', 'จังหวัดขอนแก่น', '40000']
28/20 หมู่ที่ 9 ตำบล เชียงรากน้อย อำเภอบางปะอิน จังหวัดพระนครศรีอยุธยา 13180
['28/20', 'หมู่ที่', '9', 'ตำบล', 'เชียงรากน้อย', 

In [12]:
df_json =  pd.read_json(f'../trash/familymart{str(now.date())}.json')
df_json.to_excel(f'../excel/FamilyMart/familymart{str(now.date())}.xlsx')
df_json.to_csv(f'../csv/FamilyMart/familymart{str(now.date())}.csv')

In [ ]:
#familymart
fm = pd.read_csv(f'../csv/Familymart/familymart{str(now.date())}.csv',header=0)
engine = create_engine('mysql://root:@localhost:3306/familymart')
with engine.connect() as conn, conn.begin():
    fm.to_sql(f'familymart{str(now.date())}', conn, if_exists='append', index=False)
conn.close()